Import files

# **EDSA Sentiment analysis**

**by**

# **Cameron Pike**

# **Introduction**

This notebook will provide insight into the factors and opinions surrounding climate change. 
I created a Machine Learning model that is able to classify whether or not a person believes in climate change, based on their tweet data.

In [ ]:
# utilities
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import model_selection, naive_bayes, svm

import nltk
from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import urllib

from nltk.tokenize import word_tokenize as WordTokenizer


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Import files**

Import the files into dataframes

In [ ]:

df = pd.read_csv('../input/edsa-sentiment-classification/train.csv')
df_test = pd.read_csv('../input/edsa-sentiment-classification/test.csv')
sample = pd.read_csv('../input/edsa-sentiment-classification/sample_submission.csv')


# **Exploratory Data Analysis**

View the first 5 rows of the training data

In [ ]:
print(df.head())

View the first 5 rows of the test data

In [ ]:
print(df_test.head())

View the first 5 rows of the sample data

In [ ]:
print(sample.head())

Drop the last column of the sample data

In [ ]:
#Drop sentiment column in sample. We will add the predicted sentiment later
sample = sample.iloc[: , :-1]
print(sample.head())

Analyse the columns, shape and data types in the training data

In [ ]:
print("Columns in train")
print(df.columns)

print("Shape of train")
print(df.shape)

print("Info of train")
print(df.info())

print("Data types in train")
print(df.dtypes)

#np.sum(df.isnull().any(axis=1))
print('Count of columns in the data is:  ', len(df.columns))
print('Count of rows in the data is:  ', len(df))


print("Unique sentiment values in train")
print(df['sentiment'].unique())


print("Total Unique sentiment values in train")
#Total uniques
print(df['sentiment'].nunique())




Analyse the columns, shape and data types in the test data

In [ ]:
print("Columns in test")
print(df_test.columns)

print("Shape of test")
print(df_test.shape)

print("Info of test")
print(df_test.info())

print("Data types in test")
print(df_test.dtypes)

#np.sum(df.isnull().any(axis=1))
print('Count of columns in the data is:  ', len(df_test.columns))
print('Count of rows in the data is:  ', len(df_test))




Data preprocessing,

Place the training data columns into a list

In [ ]:
# Storing data in lists.

message, sentiment = list(df['message']), list(df['sentiment'])

data=df[['message','sentiment']]
#print("data")
#print(data)
data_test=df_test[['message']]
#print("data_test")
#print(data_test)


# **Clean the data**

In [ ]:

######################################################################################
#Clean data
#####################################################################################


#Convert upper case to lower case for training data
data['message']=data['message'].str.lower()
print("before stop words")
print(data['message'].tail())

#Convert upper case to lower case for test data
data_test['message']=data_test['message'].str.lower()
print("before stop words test data")
print(data_test['message'].tail())


###############################################
#Clean punctuation
###############################################

#Remove punctuation eg !?

english_punctuations = string.punctuation
punctuations_list = english_punctuations


def cleaning_punctuations(message):
    translator = str.maketrans('', '', punctuations_list)
    return message.translate(translator)
data['message']= data['message'].apply(lambda x: cleaning_punctuations(x))
print("punctuation")
print(data['message'].tail())

data_test['message']= data_test['message'].apply(lambda x: cleaning_punctuations(x))
print("punctuation test data")
print(data_test['message'].tail())




###############################################
#Clean repeating characters
###############################################


def cleaning_repeating_char(message):
    return re.sub(r'(.)1+', r'1', message)
data['message'] = data['message'].apply(lambda x: cleaning_repeating_char(x))
print("after repeating charss")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: cleaning_repeating_char(x))
print("after repeating chars test data")
print(data_test['message'].tail())



###############################################
#Clean url's
###############################################


def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
data['message'] = data['message'].apply(lambda x: cleaning_URLs(x))
print("after clean urls")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: cleaning_URLs(x))
print("after clean urls test data")
print(data_test['message'].tail())



###############################################
#Clean repeating numbers
###############################################


def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
data['message'] = data['message'].apply(lambda x: cleaning_numbers(x))
print("after clean numbers")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: cleaning_numbers(x))
print("after clean numbers test data")
print(data_test['message'].tail())



###############################################
#Clean hashtags
###############################################


def cleaning_hastags(data):
    return re.sub('#[A-Za-z0-9_]+', '', data)
data['message'] = data['message'].apply(lambda x: cleaning_hastags(x))
print("after clean hastags")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: cleaning_hastags(x))
print("after clean hastags test data")
print(data_test['message'].tail())



###############################################
#Clean mentions
###############################################


def cleaning_mentions(data):
    return re.sub('@[A-Za-z0-9_]+', '', data)
data['message'] = data['message'].apply(lambda x: cleaning_mentions(x))
print("after clean mentions")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: cleaning_mentions(x))
print("after clean mentions test data")
print(data_test['message'].tail())




Tokenization
Word tokenization is the process of splitting a large sample of text into words. This is a requirement in natural language processing tasks where each word needs to be captured and subjected to further analysis like classifying and counting them for a particular sentiment etc. 

In [ ]:
##################################################################################################
#Tokenization
#Tokenization is the process breaking complex data like paragraphs into simple units called tokens.
#
#Sentence tokenization : split a paragraph into list of sentences using sent_tokenize() method
#Word tokenization : split a sentence into list of words using word_tokenize() method
#
#Some other important terms related to word Tokenization are:
#
#Bigrams: Tokens consist of two consecutive words known as bigrams.
#
#Trigrams: Tokens consist of three consecutive words known as trigrams.
#
#Ngrams: Tokens consist of ’N’ number of consecutive words known as ngrams
#
#
###################################################################################################


data['message'] = data['message'].astype(str).apply(WordTokenizer)
print("after tokenise")
print(data['message'])

data_test['message'] = data_test['message'].astype(str).apply(WordTokenizer)
print("after tokenise test")
print(data_test['message'])

Stopwords
Data Cleaning plays important role in NLP to remove noise from data.
Stopwords : refers to the most common words in a language (such as “the”, “a”, “an”, “in”)
which helps in formation of sentence to make sense, but these words does not provide 
any significance in language processing so remove it .

In [ ]:
###################################################################################################
#Stopwords
#
#Data Cleaning plays important role in NLP to remove noise from data.
#Stopwords : refers to the most common words in a language (such as “the”, “a”, “an”, “in”)
#which helps in formation of sentence to make sense, but these words does not provide 
#any significance in language processing so remove it .
#
#You can check list of stopwords by running below code snippet :
###################################################################################################



stop_words = set(stopwords.words("english"))

def cleaning_stopwords(message):  
    return " ".join([word for word in str(message).split() if word not in stop_words])

data['message'] = data['message'].apply(lambda message: cleaning_stopwords(message))
print("after stop words")
print(data['message'].head(20))

data_test['message'] = data_test['message'].apply(lambda message: cleaning_stopwords(message))
print("after stop words test data")
print(data_test['message'].head(20))

PorterStemmer
Stemming is the process of producing morphological variants of a root/base word.
Stemming programs are commonly referred to as stemming algorithms or stemmers.
A stemming algorithm reduces the words “chocolates”, “chocolatey”, and “choco” to the root word, “chocolate”


In [ ]:

###################################################################################################
#PorterStemmer
#
#Stemming is the process of producing morphological variants of a root/base word.
#Stemming programs are commonly referred to as stemming algorithms or stemmers.
#A stemming algorithm reduces the words “chocolates”, “chocolatey”, and “choco” to the root
#word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve
###################################################################################################


st = nltk.PorterStemmer()

def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data

data['message']= data['message'].apply(lambda x: stemming_on_text(x))
print("after porter stemmer")
print(data['message'].tail())

data_test['message']= data_test['message'].apply(lambda x: stemming_on_text(x))
print("after porter stemmer test data")
print(data_test['message'].tail())


Lemmatization
Major drawback of stemming is it produces Intermediate representation of word. Stemmer may or may not return meaningful word.

To overcome this problem Lemmatization comes into picture.

Stemming algorithm works by cutting suffix or prefix from the word.On the contrary Lemmatization consider morphological analysis of the words and returns meaningful word in proper form.

Hence,lemmatization is preferred.

In [ ]:

#######################################################################################################
#Lemmatization
#Major drawback of stemming is it produces Intermediate representation of word. Stemmer may or may not return meaningful word.
#
#To overcome this problem Lemmatization comes into picture.
#
#Stemming algorithm works by cutting suffix or prefix from the word.On the contrary Lemmatization consider morphological analysis of the words and returns meaningful word in proper form.
#
#Hence,lemmatization is preferred.
#########################################################################################################


lm = nltk.WordNetLemmatizer()

def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['message'] = data['message'].apply(lambda x: lemmatizer_on_text(x))
print("after lemmatizer")
print(data['message'].tail())

data_test['message'] = data_test['message'].apply(lambda x: lemmatizer_on_text(x))
print("after lemmatizer test data")
print(data_test['message'].tail())

# **Modeling**

TF-IDF stands for Term Frequency-Inverse Document Frequency
“Term frequency–inverse document frequency, is a numerical statistic that is intended to 
reflect how important a word is to a document in a collection or corpus.”

Term Frequency: is a scoring of the frequency of the word in the current document.
Inverse Document Frequency: is a scoring of how rare the word is across documents.
TF-IDF model contains information on the more important words and the less important ones as well.


In [ ]:
###########################################################################################
#TF-IDF stands for Term Frequency-Inverse Document Frequency
#“Term frequency–inverse document frequency, is a numerical statistic that is intended to 
#reflect how important a word is to a document in a collection or corpus.”
#
#Term Frequency: is a scoring of the frequency of the word in the current document.
#Inverse Document Frequency: is a scoring of how rare the word is across documents.
#TF-IDF model contains information on the more important words and the less important ones as well.
#
#Note:In order to use TF-IDF we must pass Strings instead of tokens.
#we can use detokenizer or join method to join all tokens of list into a single string
#################################################################################################


#Split data into X and Y to create seperate target
X=data.message
y=data.sentiment
competion_test=data_test.message



# Separating the data for training data and for testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =42)



vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))


print('X_train before vectoriser.transform X_train')
print(X_train)

X_train = vectoriser.transform(X_train)
print('X_train after vectoriser.transform X_train')
print(X_train)

X_test  = vectoriser.transform(X_test)
print('X_test after vectoriser.transform X_test')
print(X_test)

#Our sample from competition to predict on
X_test_sample  = vectoriser.transform(competion_test)
print('X_test_sample after vectoriser.transform data_test')
print(X_test_sample)

Performance evaluation of models

In [ ]:
##########################################################################
#Train and evaluate models
#################################################################


def model_Evaluate(model):
 # Predict values for Test dataset
 y_pred = model.predict(X_test)
 print("y predict")
 print(y_pred)
 # Print the evaluation metrics for the dataset.
 print(classification_report(y_test, y_pred))
 # Compute and plot the Confusion matrix
 cf_matrix = confusion_matrix(y_test, y_pred)

    
#BernoulliNB
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
print("score for BNB")
model_Evaluate(BNBmodel)

y_pred_final_BNB = BNBmodel.predict(X_test_sample)
print("y predict final BNB")
print(y_pred_final_BNB)

#Add extra column to output
#Copy the sample file
sample_BNB_copy = sample.copy()
sample_BNB_copy['sentiment'] = y_pred_final_BNB
sample_BNB_copy[['tweetid','sentiment']].to_csv('testsubmission_BNB.csv', index=False)


#LinearSVC
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
print("score for svc")
model_Evaluate(SVCmodel)

y_pred_final_SVC = SVCmodel.predict(X_test_sample)
print("y predict final SVC")
print(y_pred_final_SVC)

#Add extra column to output
#Copy the sample file
sample_SVC_copy = sample.copy()
sample_SVC_copy['sentiment'] = y_pred_final_SVC
sample_SVC_copy[['tweetid','sentiment']].to_csv('testsubmission_SVC.csv', index=False)



#Logistic Regression
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
print("score for LRmodel")
model_Evaluate(LRmodel)

y_pred_final_LR = LRmodel.predict(X_test_sample)
print("y predict final LR")
print(y_pred_final_LR)

#Add extra column to output
#Copy the sample file
sample_LR_copy = sample.copy()
sample_LR_copy['sentiment'] = y_pred_final_LR
sample_LR_copy[['tweetid','sentiment']].to_csv('testsubmission_LR.csv', index=False)




#RandomForestClassifier()
RFCmodel = RandomForestClassifier()
RFCmodel.fit(X_train, y_train)
print("score for RFCmodel")
model_Evaluate(RFCmodel)

y_pred_final_RFC = RFCmodel.predict(X_test_sample)
print("y predict final RFC")
print(y_pred_final_RFC)

#Add extra column to output
#Copy the sample file
sample_RFC_copy = sample.copy()
sample_RFC_copy['sentiment'] = y_pred_final_RFC
sample_RFC_copy[['tweetid','sentiment']].to_csv('testsubmission_RFC.csv', index=False)


Visualiztions

In [ ]:
############################################################################
#Graphs
##########################################################################


sns.countplot(x='sentiment', data=df)



#data_neg = data['message'][:500]
data_neg = X[:15818]
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 500 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(data_neg))
plt.imshow(wc)





# set up label dataframe for future refrences
'''
label = [-1, 0, 1, 2]
labelN = ["Against", "Neutral", "Pro", "FactualNews"]
labelDesc = [
    "the tweet does not believe in man-made climate change"
    , "the tweet neither supports nor refutes the belief of man-made climate change"
    , "the tweet supports the belief of man-made climate change"
    , "the tweet links to factual news about climate change"
]

dfLabel = pd.DataFrame(list(zip(label, labelN, labelDesc)), columns=["label", "sentiment", "description"])
'''
print(df.head())

plt.figure(figsize = (7, 7))
plt.pie(df.sentiment.value_counts().values, labels = df.sentiment.value_counts().index, autopct = '%2.1f%%', textprops={'fontsize': 15})
plt.title('Sentiment Distribution of the Tweet Dataset', fontsize=20)
plt.tight_layout()
plt.show()
